# Final Results: Facebook Likes

Import libraries

In [2]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression as LogReg
import matplotlib
import matplotlib.pyplot as plt
import math
import StringIO
import sklearn
from sklearn.decomposition import PCA
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn import svm
from sklearn.dummy import DummyClassifier as Dummy
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LinearRegression as LinReg
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier as DecisionTree
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.svm import SVC
%matplotlib inline

/Users/kyledevine/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
movie_data = pd.read_csv('movie_metadata.csv')
movie_data = movie_data[np.isfinite(movie_data['budget'])]
movie_data = movie_data[np.isfinite(movie_data['duration'])]
movie_data = movie_data[np.isfinite(movie_data['gross'])]
movie_data = movie_data[np.isfinite(movie_data['title_year'])]
movie_data = movie_data[np.isfinite(movie_data['cast_total_facebook_likes'])]
movie_data = movie_data[np.isfinite(movie_data['imdb_score'])]

# create genre list
genre_list = ['Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Drama', 'Fantasy', 'History', 'Horror', 'Mystery', 'Romance']

# create categorical variables for these common genres
for genre in genre_list:
    lst = []
    for index, row in movie_data.iterrows():
        if (row['genres']).find(genre) != -1:
            lst.append(1)
        else:
            lst.append(0)
    movie_data[genre] = lst

In [4]:
counts = pd.value_counts(movie_data['director_name'])
counts = counts[counts > 6]

directors = counts.index.tolist()
print "Number of Directors: ", len(directors)

Number of Directors:  91


In [5]:
counts_actors = pd.value_counts(movie_data['actor_1_name'])

counts_actors = counts_actors[counts_actors > 7]

actors = counts_actors.index.tolist()
print "Number of Actors: ",len(actors)

Number of Actors:  97


In [6]:
for director in directors:
    lst = []
    for _, row in movie_data.iterrows():
        if row['director_name'] == director:
            lst.append(1)
        else:
            lst.append(0)
    movie_data[director] = lst

In [7]:
for actor in actors:
    lst = []
    for _, row in movie_data.iterrows():
        if row['actor_1_name'] == actor or row['actor_2_name'] == actor or row['actor_3_name'] == actor:
            lst.append(1)
        else:
            lst.append(0)
    movie_data[actor] = lst

In [8]:
budget_inf = []
gross_inf = []

# take log value of each budget and gross value
for _, row in movie_data.iterrows():
    budget = math.log10(row['budget'])
    gross = math.log10(row['gross'])
    budget_inf.append(budget)
    gross_inf.append(gross)

# add as columns of dataframe
movie_data['budget_inf'] = budget_inf
movie_data['gross_inf'] = gross_inf

In [9]:
# create classes for facebook likes
lst = []
for index, row in movie_data.iterrows():
        if row['movie_facebook_likes'] > 150000:
            lst.append(5)
        elif row['movie_facebook_likes'] > 80000:
            lst.append(4)
        elif row['movie_facebook_likes'] > 20000:
            lst.append(3)
        elif row['movie_facebook_likes'] > 7500: 
            lst.append(2)
        else:
            lst.append(1)

movie_data['popularity_class'] = lst

In [11]:
# score SVC model for facebook likes class (with budget)
train_test = pd.DataFrame({})

for genre in genre_list:
    train_test = pd.concat([train_test, movie_data[genre]], axis = 1)

train_test = pd.concat([train_test, movie_data[directors]], axis = 1)
train_test = pd.concat([train_test, movie_data[actors]], axis = 1)
train_test = pd.concat([train_test, movie_data['budget']], axis = 1)
train_test = pd.concat([train_test, movie_data['title_year']], axis = 1)
train_test = pd.concat([train_test, movie_data['popularity_class']], axis = 1)
train_test = pd.concat([train_test, pd.get_dummies(movie_data['content_rating'])], axis = 1)

# split
training, testing = sklearn.model_selection.train_test_split(train_test, train_size = 0.4, test_size = 0.6)

In [12]:
# fit and score dummy and RFC model
print "Popularity class (with Budget)"
d_model = Dummy()
d_model.fit(training.drop('popularity_class', axis=1), training['popularity_class'])
print "\tDummy Model: ", d_model.score(testing.drop('popularity_class', axis = 1), testing['popularity_class'])

rfc_model = RFC(max_depth = 10, n_estimators=50)
rfc_model.fit(training.drop('popularity_class', axis=1), training['popularity_class'])

print "\tRandom Forest Classifier Model", rfc_model.score(testing.drop('popularity_class', axis = 1), testing['popularity_class'])

Popularity class (with Budget)
	Dummy Model:  0.546272493573
	Random Forest Classifier Model 0.74764353042
